In [1]:
import time
import json
import pandas as pd
import pymysql
from bs4 import BeautifulSoup
from selenium import webdriver

# --- 1️⃣ 크롬 드라이버 설정 ---
def CoffeeBean_store():
    CoffeeBean_URL = "https://www.coffeebeankorea.com/store/store.asp"
    wd = webdriver.Chrome()

    result = []

    for i in range(1, 10):  # 마지막 매장번호(최신 매장까지 크롤링),1~395
        wd.get(CoffeeBean_URL)
        time.sleep(1)  # 웹페이지 연결 대기

        try:
            wd.execute_script(f"storePop2({i})")
            time.sleep(3)  # 스크립트 실행 대기
            html = wd.page_source
            soupCB = BeautifulSoup(html, 'html.parser')

            # 매장명 가져오기
            store_name_h2 = soupCB.select("div.store_txt > h2")
            store_name = store_name_h2[0].string.strip() if store_name_h2 else "정보 없음"

            # 주소 및 전화번호 가져오기
            store_info = soupCB.select("div.store_txt > table.store_table > tbody > tr > td")
            store_address = store_info[2].text.strip() if len(store_info) > 2 else "주소 없음"
            store_phone = store_info[3].text.strip() if len(store_info) > 3 else "전화번호 없음"

            print(f"📍 매장명: {store_name} | 📍 주소: {store_address} | 📞 전화번호: {store_phone}")

            result.append({"store": store_name, "address": store_address, "phone": store_phone})

        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            continue

    wd.quit()
    return result


# --- 2️⃣ JSON 파일 저장 ---
def save_to_json(data):
    json_file = "CoffeeBean.json"
    with open(json_file, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"✅ JSON 파일 저장 완료: {json_file}")


# --- 3️⃣ MariaDB(MySQL) 저장 ---
def save_to_mariadb(data):
    db = pymysql.connect(
        host="localhost",
        user="webuser",  # 변경 필요
        password="webuser",  # 변경 필요
        database="webdb"
    )
    cursor = db.cursor()

    # 테이블 생성 (처음 실행 시 필요)
    create_table_query = """
    CREATE TABLE IF NOT EXISTS coffeebean_stores (
        id INT AUTO_INCREMENT PRIMARY KEY,
        store VARCHAR(255) NOT NULL,
        address TEXT NOT NULL,
        phone VARCHAR(50)
    )
    """
    cursor.execute(create_table_query)

    # 데이터 저장 쿼리
    insert_query = """
    INSERT INTO coffeebean_stores (store, address, phone)
    VALUES (%s, %s, %s)
    """

    try:
        cursor.executemany(insert_query, [(d["store"], d["address"], d["phone"]) for d in data])
        db.commit()
        print(f"✅ {len(data)}개의 매장 데이터가 MariaDB에 저장되었습니다.")
    except Exception as e:
        db.rollback()
        print(f"❌ 데이터 저장 중 오류 발생: {e}")

    cursor.close()
    db.close()

In [2]:
# --- 4️⃣ 실행 (메인 함수) ---
def main():
    print("🚀 CoffeeBean 매장 정보 크롤링 시작...")
    store_data = CoffeeBean_store()

    print("\n💾 JSON 파일 저장 중...")
    save_to_json(store_data)

    print("\n📂 MariaDB 저장 중...")
    save_to_mariadb(store_data)

    print("\n🎉 모든 작업 완료!")


if __name__ == '__main__':
    main()

🚀 CoffeeBean 매장 정보 크롤링 시작...
📍 매장명: 정보 없음 | 📍 주소: 주소 없음 | 📞 전화번호: 전화번호 없음
📍 매장명: 정보 없음 | 📍 주소: 주소 없음 | 📞 전화번호: 전화번호 없음
📍 매장명: 차병원점 | 📍 주소: 서울시 강남구 논현로 566 강남차병원1층 | 📞 전화번호: 02-538-7615
📍 매장명: 정보 없음 | 📍 주소: 주소 없음 | 📞 전화번호: 전화번호 없음
📍 매장명: 정보 없음 | 📍 주소: 주소 없음 | 📞 전화번호: 전화번호 없음
📍 매장명: 강남대로점 | 📍 주소: 서울시 서초구 강남대로 369 1층 | 📞 전화번호: 02-588-5778
📍 매장명: 정보 없음 | 📍 주소: 주소 없음 | 📞 전화번호: 전화번호 없음
📍 매장명: 정보 없음 | 📍 주소: 주소 없음 | 📞 전화번호: 전화번호 없음
📍 매장명: 정보 없음 | 📍 주소: 주소 없음 | 📞 전화번호: 전화번호 없음

💾 JSON 파일 저장 중...
✅ JSON 파일 저장 완료: CoffeeBean.json

📂 MariaDB 저장 중...
✅ 9개의 매장 데이터가 MariaDB에 저장되었습니다.

🎉 모든 작업 완료!
